In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("msambare/fer2013")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/fer2013


In [2]:
# 1. Install system packages
!apt-get install -y cmake
!apt-get install -y libboost-all-dev

# 2. Install dlib
!pip install dlib

# 3. Install face_recognition
!pip install face_recognition

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libboost-all-dev is already the newest version (1.74.0.3ubuntu7).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566166 sha256=0da45419cde7c2e70828ee84ec0a2bfc4a14deade83583af0aa8c9a6e84d14b1
  Stored in directory: /root/.cache/pip/wheels/04/52/ec/9355da79c29f160b038a20c784db2803c2f9fa2c8a462c176a
Successfully built face-recognition-models


In [9]:
import os
import json
import torch
import random
from PIL import Image
from transformers import AutoImageProcessor, ViTForImageClassification
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
from torch.nn.functional import softmax


emotions = ['happy', 'sad', 'fear', 'neutral', 'surprise', 'angry', 'disgust']
emotion_to_index = {e: i for i, e in enumerate(emotions)}

label_map = {
    'angry': 'angry',
    'anger': 'angry',
    'fear': 'fear',
    'scared': 'fear',
    'sadness': 'sad',
    'sad': 'sad',
    'disgust': 'disgust',
    'happy': 'happy',
    'happiness': 'happy',
    'surprise': 'surprise',
    'suprise': 'surprise',
    'neutral': 'neutral',
    'contempt': 'neutral'
}

def one_hot(label):
    vec = [0] * 7
    vec[emotion_to_index[label]] = 1
    return vec

def load_face(image_path):
    try:
        img = Image.open(image_path).convert("RGB").resize((224, 224))
        return img
    except Exception as e:
        print(f"Failed to load {image_path}: {e}")
        return None

In [10]:
MODELS = {
    '1': 'dima806/facial_emotions_image_detection',
    '2': 'trpakov/vit-face-expression',
    '3': 'motheecreator/vit-Facial-Expression-Recognition',
}
dataset_dir = "/kaggle/input/fer2013/test"

# Setup device
device = "cuda" if torch.cuda.is_available() else "cpu"
device = "mps" if torch.backends.mps.is_available() else device

processors = {}
models = {}

for key, name in MODELS.items():
    processor = AutoImageProcessor.from_pretrained(name)
    model = ViTForImageClassification.from_pretrained(name).to(device)
    processors[key] = processor
    models[key] = model

model.safetensors:   0%|          | 0.00/343M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/325 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/915 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/343M [00:00<?, ?B/s]

In [11]:
samples = []

def get_first_70_percent_per_class(dataset_dir):
    training_samples = []
    for label in sorted(os.listdir(dataset_dir)):
        folder = os.path.join(dataset_dir, label)
        if not os.path.isdir(folder):
            continue
        image_paths = sorted(os.listdir(folder))
        split_idx = int(len(image_paths) * 0.7)
        for fname in image_paths[:split_idx]:
            training_samples.append((os.path.join(folder, fname), label))
    return training_samples

training_samples = get_first_70_percent_per_class(dataset_dir)

for path, label in training_samples:
    true_label = label_map.get(label.lower(), label.lower())
    face = load_face(path)
    if face is None:
        continue

    features_per_model = []
    max_confidences = []
    pred_labels = []

    for processor, model in zip(processors.values(), models.values()):
        inputs = processor(images=face, return_tensors="pt").to(model.device)
        with torch.no_grad():
            outputs = model(**inputs)
        probs = softmax(outputs.logits, dim=-1).squeeze().cpu().tolist()
        features_per_model.append(probs)
        max_confidences.append(max(probs))
        pred_label = model.config.id2label[probs.index(max(probs))].lower()
        pred_labels.append(label_map.get(pred_label, pred_label))

    # Agreement logic
    agree_3 = int(pred_labels.count(pred_labels[0]) == 3)
    agree_2 = int(len(set(pred_labels)) == 2)

    # Build final feature vector: 21 softmax + 3 max_conf + 2 agreement = 26
    features = []
    for probs in features_per_model:
        features.extend(probs)
    features.extend(max_confidences)
    features.append(agree_2)
    features.append(agree_3)

    samples.append({
        "features": features,
        "true_label": true_label
    })

# Save dataset
with open("votingnet_training_data.json", "w") as f:
    json.dump(samples, f, indent=2)

print(f"Saved {len(samples)} training samples for VotingNet with 26-dim input.")

Saved 5020 training samples for VotingNet with 26-dim input.


In [32]:
from sklearn.model_selection import train_test_split
from collections import Counter

# Load updated dataset with 26-dim features
with open("votingnet_training_data.json", "r") as f:
    data = json.load(f)

X = torch.tensor([d["features"] for d in data], dtype=torch.float32)
y = torch.tensor([emotion_to_index[d["true_label"]] for d in data], dtype=torch.long)

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

class VotingNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(26, 384)
        self.norm1 = nn.LayerNorm(384)
        self.fc2 = nn.Linear(384, 192)
        self.fc3 = nn.Linear(192, 64)
        self.dropout = nn.Dropout(0.3)
        self.out = nn.Linear(64, 7)

    def forward(self, x):
        x = F.relu(self.norm1(self.fc1(x)))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        return self.out(x)

class FocalLoss(nn.Module):
    def __init__(self, alpha=1.0, gamma=2.0):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce_loss)
        return self.alpha * ((1 - pt) ** self.gamma * ce_loss).mean()

model = VotingNet().to(device)
criterion = FocalLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.7)

# Training loop with validation
best_val_acc = 0
for epoch in range(50):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train.to(device))
    loss = criterion(outputs, y_train.to(device))
    loss.backward()
    optimizer.step()
    scheduler.step()

    model.eval()
    with torch.no_grad():
        train_acc = (outputs.argmax(1) == y_train.to(device)).float().mean().item() * 100
        val_outputs = model(X_val.to(device))
        val_acc = (val_outputs.argmax(1) == y_val.to(device)).float().mean().item() * 100

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "votingnet_model.pt")

    if epoch % 10 == 0 or epoch == 49:
        print(f"Epoch {epoch}: Loss={loss.item():.4f}, Train Acc={train_acc:.2f}%, Val Acc={val_acc:.2f}%")

print(f"Best validation accuracy: {best_val_acc:.2f}%")
print("Saved best VotingNet model to 'votingnet_model.pt'")

Epoch 0: Loss=1.4236, Train Acc=19.70%, Val Acc=26.20%
Epoch 10: Loss=0.4198, Train Acc=86.23%, Val Acc=86.45%
Epoch 20: Loss=0.2152, Train Acc=89.19%, Val Acc=88.15%
Epoch 30: Loss=0.1862, Train Acc=90.84%, Val Acc=90.24%
Epoch 40: Loss=0.1701, Train Acc=91.61%, Val Acc=90.34%
Epoch 49: Loss=0.1579, Train Acc=91.81%, Val Acc=90.44%
Best validation accuracy: 90.44%
Saved best VotingNet model to 'votingnet_model.pt'


In [33]:
def get_last_30_percent(dataset_dir):
    all_test_samples = []
    for label in sorted(os.listdir(dataset_dir)):
        folder = os.path.join(dataset_dir, label)
        if not os.path.isdir(folder):
            continue
        image_paths = sorted(os.listdir(folder))
        total = len(image_paths)
        split_idx = int(total * 0.7)
        for fname in image_paths[split_idx:]:
            all_test_samples.append((os.path.join(folder, fname), label))
    return all_test_samples

def evaluate_model(processor, model, test_samples):
    total, correct = 0, 0
    per_label_stats = {}

    # Track all predictions and true labels
    all_preds = []
    all_labels = []

    for path, label in test_samples:
        true_label = label_map.get(label.lower(), label.lower())
        face = load_face(path)
        if face is None:
            continue

        inputs = processor(images=face, return_tensors="pt").to(model.device)
        with torch.no_grad():
            outputs = model(**inputs)
        pred = outputs.logits.argmax(-1).item()
        pred_label = label_map.get(model.config.id2label[pred].lower(), model.config.id2label[pred].lower())

        all_preds.append(pred_label)
        all_labels.append(true_label)

        total += 1
        if true_label not in per_label_stats:
            per_label_stats[true_label] = [0, 0]
        per_label_stats[true_label][1] += 1
        if pred_label == true_label:
            correct += 1
            per_label_stats[true_label][0] += 1

    print(f"Evaluated {total} images for model.")
    print("Accuracy per emotion:")
    for label, (corr, tot) in per_label_stats.items():
        acc = (corr / tot * 100) if tot > 0 else 0
        print(f"  {label}: {acc:.2f}% ({corr}/{tot})")

    # At the end
    le = LabelEncoder().fit(emotions)
    f1 = f1_score(le.transform(all_labels), le.transform(all_preds), average='macro')
    print(f"F1 Score (macro): {f1:.4f}")
    return correct / total * 100 if total > 0 else 0

def evaluate_combined(models, processors, test_samples):
    total, correct = 0, 0
    per_label_stats = {}

    # Track all predictions and true labels
    all_preds = []
    all_labels = []

    model_voting = VotingNet().to(device)
    model_voting.load_state_dict(torch.load("votingnet_model.pt", map_location=device, weights_only=True))
    model_voting.eval()

    for path, label in test_samples:
        true_label = label_map.get(label.lower(), label.lower())
        face = load_face(path)
        if face is None:
            continue

        features_per_model = []
        pred_labels = []
        max_confidences = []

        for processor, model in zip(processors, models):
            inputs = processor(images=face, return_tensors="pt").to(model.device)
            with torch.no_grad():
                outputs = model(**inputs)
            probs = torch.nn.functional.softmax(outputs.logits, dim=-1).squeeze().cpu().tolist()
            features_per_model.append(probs)
            max_confidences.append(max(probs))
            pred_label = model.config.id2label[probs.index(max(probs))].lower()
            pred_labels.append(label_map.get(pred_label, pred_label))

        # Agreement logic
        agree_3 = int(pred_labels.count(pred_labels[0]) == 3)
        agree_2 = int(len(set(pred_labels)) == 2)

        # Build input feature vector
        features = []
        for probs in features_per_model:
            features.extend(probs)
        features.extend(max_confidences)
        features.append(agree_2)
        features.append(agree_3)

        input_tensor = torch.tensor([features], dtype=torch.float32).to(device)

        with torch.no_grad():
            output = model_voting(input_tensor)
        final_pred = emotions[output.argmax().item()]

        # In loop after final_pred:
        all_preds.append(final_pred)
        all_labels.append(true_label)

        total += 1
        if true_label not in per_label_stats:
            per_label_stats[true_label] = [0, 0]
        per_label_stats[true_label][1] += 1
        if final_pred == true_label:
            correct += 1
            per_label_stats[true_label][0] += 1

    print(f"\nEvaluated {total} images using VotingNet.")
    print("Accuracy per emotion:")
    for label, (corr, tot) in per_label_stats.items():
        acc = (corr / tot * 100) if tot > 0 else 0
        print(f"  {label}: {acc:.2f}% ({corr}/{tot})")

    # At the end
    le = LabelEncoder().fit(emotions)
    f1 = f1_score(le.transform(all_labels), le.transform(all_preds), average='macro')
    print(f"F1 Score (macro): {f1:.4f}")

    return correct / total * 100 if total > 0 else 0


# Main function
def main(dataset_dir):
    print(f"Evaluating on device: {device}")

    print("\nLoading models...")
    processors = {}
    models = {}
    for key, name in MODELS.items():
        processor = AutoImageProcessor.from_pretrained(name)
        model = ViTForImageClassification.from_pretrained(name).to(device)
        processors[key] = processor
        models[key] = model

    test_samples = get_last_30_percent(dataset_dir)

    print("\nEvaluating individual models:")
    for key, model_id in MODELS.items():
        acc = evaluate_model(processors[key], models[key], test_samples)
        print(f"Model {key} ({model_id}): {acc:.2f}% accuracy")

    print("\nEvaluating combined (VotingNet) model:")
    acc = evaluate_combined(list(models.values()), list(processors.values()), test_samples)
    print(f"Combined model: {acc:.2f}% accuracy")

# Run
main("/kaggle/input/fer2013/test")

Evaluating on device: cuda

Loading models...

Evaluating combined (VotingNet) model:

Evaluated 2158 images using VotingNet.
Accuracy per emotion:
  angry: 88.89% (256/288)
  disgust: 97.06% (33/34)
  fear: 89.29% (275/308)
  happy: 93.81% (500/533)
  neutral: 90.54% (335/370)
  sad: 90.67% (340/375)
  surprise: 94.40% (236/250)
F1 Score (macro): 0.9222
Combined model: 91.52% accuracy
